#### Test3: 
Este archivo tiene por finalidad mostrar el flujo completo del preprocesamiento de los parches y una eventual entrada en la red. Para más detalle, visitar el archivo [A1_guiaNC.ipynb](A1_guiaNC.ipynb)

In [3]:
import xarray as xr
import tensorflow as tf
import numpy as np
import glob
import os
import numpy as np
from tqdm import tqdm
import B1_readNCdata as nc
import B2_Model as cnn

2.11.0


#### 1-. Instanciamiento de las clases, seteo de atributos importantes y primeros pasos:

In [5]:
#target
era5 = nc.NCFile('era5-land_*.nc', path='./Data/observed/2006/')
era5.setVar('t2m')
era5.loadVar()

#input
g05 = nc.NCFile('GFS0.5_t2m_heightAboveGround_instant_200610*.nc', path='./Data/forecasted/2006/2006101000/')
g05.setVar('t2m')
g05.loadVar()

g05.upscaleVar(era5.values)

c:\Users\ruben\anaconda3\envs\dwn_104\lib\site-packages\xarray\backends\plugins.py:71: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


<xarray.DataArray 't2m' (valid_time: 40, latitude: 401, longitude: 101)>
dask.array<copy, shape=(40, 401, 101), dtype=float32, chunksize=(40, 401, 101), chunktype=numpy.ndarray>
Coordinates:
    time               datetime64[ns] ...
    heightAboveGround  float64 ...
  * latitude           (latitude) float32 -15.0 -15.1 -15.2 ... -54.9 -55.0
  * longitude          (longitude) float32 -75.0 -74.9 -74.8 ... -65.1 -65.0
  * valid_time         (valid_time) datetime64[ns] 2006-10-10 ... 2006-10-14T...
Attributes: (12/29)
    GRIB_paramId:                             167
    GRIB_dataType:                            fc
    GRIB_numberOfPoints:                      259920
    GRIB_typeOfLevel:                         heightAboveGround
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_name:                                2 metre temperature
    GRIB_shortName:                           2t
    GRIB_units:                               K
    long_name:                                2 metre temperature
    units:                                    K
    standard_name:                            air_temperature

#### 2-. Preparación de los parches

Comentario: Ejecutar la siguiente celda en caso de querer crear los parches nuevamente.

In [4]:
inp = g05.extraccion_parches(Var=g05.getVar())
target = era5.extraccion_parches(Var=era5.getVar())
lat = era5.sampleLat()
lon = era5.sampleLon()
orog_np = era5.sampleBand1()
time1,time2 = era5.sampleDayOfYear()

100%|██████████| 38/38 [03:03<00:00,  4.84s/it]


Stack patches shape:  (1368, 3, 32, 32, 1)


100%|██████████| 38/38 [00:09<00:00,  3.90it/s]


Stack patches shape:  (1368, 3, 32, 32, 1)


100%|██████████| 38/38 [00:04<00:00,  7.61it/s]


...

Stack patches shape:  (1368, 3, 32, 32, 1)


100%|██████████| 38/38 [00:08<00:00,  4.70it/s]


Stack patches shape:  (1368, 3, 32, 32, 1)


100%|██████████| 38/38 [09:15<00:00, 14.62s/it]

Stack patches shape:  (95000, 3, 4, 4, 1)


Cargamos los parches previamente creados:

In [7]:
with open('C1_parches.npy', 'rb') as f:
    inp_np = tf.convert_to_tensor(np.load(f))
    target_np = tf.convert_to_tensor(np.load(f))
    lat_np = tf.convert_to_tensor(np.load(f))
    lon_np = tf.convert_to_tensor(np.load(f))
    orog_np =tf.convert_to_tensor(np.load(f))
    time1_np = tf.convert_to_tensor(np.load(f))
    time2_np = tf.convert_to_tensor(np.load(f))

In [ ]:
#comparar inputs cargados con los creados originalmente
#comp=tf.math.reduce_all(tf.equal(inp,inp_np))
#print(comp)

Dimensión de los tensores iniciales

In [8]:
print("target: ",target_np.shape)
print("inp: ",inp_np.shape)
print("lat: ",lat_np.shape)
print("long: ",lon_np.shape)
print("orog: ",orog_np.shape)
print("time1: ",time1_np.shape)
print("time2: ",time2_np.shape)

target:  (1368, 3, 32, 32, 1)
inp:  (1368, 3, 32, 32, 1)
lat:  (1368, 3, 32, 32, 1)
long:  (1368, 3, 32, 32, 1)
orog:  (1368, 3, 32, 32, 1)
time1:  (1368, 3, 32, 32, 1)
time2:  (95000, 3, 4, 4, 1)


#### 3-. Eliminación de parches nulos, normalización y división de la data:

Chequeo de la cantidad de datos Nan que contienen nuestras variables del input:

In [9]:
era5.cantidad_nan(inputs=[target_np,inp_np,lat_np,lon_np,orog_np,time1_np,time2_np])

target- num valores nan: 452542
inp- num valores nan: 0
lat- num valores nan: 0
lot- num valores nan: 0
orog- num valores nan: 0
time1- num valores nan: 0
time2- num valores nan: 0


Índices de los parches que no son completamente Nan

In [10]:
idx=era5.parches_nan_target(target_np)
print(len(idx))
idx[:10]

Cantidad de parches nulos:  152
Cantidad de parches no nulos:  1216
1216


[0, 1, 2, 4, 5, 7, 8, 10, 11, 13]

- 3.1-. Eliminación de los parches que son completamente Nan

In [11]:
target,inp,lat,long,orog,time1=era5.filtracion_nan(target_np,inp_np,lat_np,lon_np,orog_np,time1_np,idx)

Dimensión de los parches post limpieza:

In [12]:
print("target: ",target.shape)
print("inp: ",inp.shape)
print("lat: ",lat.shape)
print("long: ",long.shape)
print("orog: ",orog.shape)
print("time1: ",time1.shape)
#print("time2: ",time2.shape)

target:  (1216, 3, 32, 32, 1)
inp:  (1216, 3, 32, 32, 1)
lat:  (1216, 3, 32, 32, 1)
long:  (1216, 3, 32, 32, 1)
orog:  (1216, 3, 32, 32, 1)
time1:  (1216, 3, 32, 32, 1)


- 3.2-. Normalización de los tensores: 

In [13]:
target_norm, scale_factor_target = era5.normalization(target, for_target=True)
inp_norm, scale_factor_inp = era5.normalization(inp)
lat_norm, scale_factor_lat = era5.normalization(lat)
lon_norm, scale_factor_lon = era5.normalization(long)
orog_norm, scale_factor_orog = era5.normalization(orog)
time_norm, scale_factor_time = era5.normalization(time1)

In [14]:
era5.cantidad_nan(inputs=[target_norm,inp_norm,lat_norm,lon_norm,orog_norm,time_norm,time2_np])

target- num valores nan: 296894
inp- num valores nan: 0
lat- num valores nan: 0
lot- num valores nan: 0
orog- num valores nan: 0
time1- num valores nan: 0
time2- num valores nan: 0


In [15]:
idx_2=era5.parches_nan_target(target_norm)

Cantidad de parches nulos:  0
Cantidad de parches no nulos:  1216


- 3.3-. Segmentación de los tensores

In [16]:
(inp_tr,time_tr,orog_tr,lat_tr,lon_tr,target_tr,
 inp_va,time_va,orog_va,lat_va,lon_va,target_va,
 inp_te,time_te, orog_te,lat_te,lon_te, target_te) = era5.mysplit(inp_norm,time_norm,orog_norm,lat_norm,lon_norm,target_norm)

- Ajuste de las dimensiones para *time1*:

In [17]:
time_norm, scale_factor_time = era5.normalization(time2_np[:inp_norm.shape[0]])
time_tr =time_norm[:inp_tr.shape[0]]
time_va = time_norm[inp_tr.shape[0]:inp_tr.shape[0]+inp_va.shape[0]]
time_te = time_norm[inp_tr.shape[0]+inp_va.shape[0]:inp_norm.shape[0]]

##### Resumen división de la data:

In [18]:
print("#"*10+" "+"Entrenamiento"+" "+"#"*10)
print("target: ",target_tr.shape)
print("inp: ",inp_tr.shape)
print("lat: ",lat_tr.shape)
print("long: ",lon_tr.shape)
print("orog: ",orog_tr.shape)
print("time1: ",time_tr.shape)

########## Entrenamiento ##########
target:  (729, 1, 32, 32, 1)
inp:  (729, 3, 32, 32, 1)
lat:  (729, 3, 32, 32, 1)
long:  (729, 3, 32, 32, 1)
orog:  (729, 3, 32, 32, 1)
time1:  (729, 3, 4, 4, 1)


In [19]:
print("#"*10+" "+"Validación"+" "+"#"*10)
print("target: ",target_va.shape)
print("inp: ",inp_va.shape)
print("lat: ",lat_va.shape)
print("long: ",lon_va.shape)
print("orog: ",orog_va.shape)
print("time1: ",time_va.shape)

########## Validación ##########
target:  (243, 1, 32, 32, 1)
inp:  (243, 3, 32, 32, 1)
lat:  (243, 3, 32, 32, 1)
long:  (243, 3, 32, 32, 1)
orog:  (243, 3, 32, 32, 1)
time1:  (243, 3, 4, 4, 1)


In [20]:
print("#"*10+" "+"Testeo"+" "+"#"*10)
print("target: ",target_te.shape)
print("inp: ",inp_te.shape)
print("lat: ",lat_te.shape)
print("long: ",lon_te.shape)
print("orog: ",orog_te.shape)
print("time1: ",time_te.shape)

########## Testeo ##########
target:  (244, 1, 32, 32, 1)
inp:  (244, 3, 32, 32, 1)
lat:  (244, 3, 32, 32, 1)
long:  (244, 3, 32, 32, 1)
orog:  (244, 3, 32, 32, 1)
time1:  (244, 3, 4, 4, 1)


#### 4-. Entrenamiento de la red:

Nota: De momento no contamos con entrenamiento de la red debido a que no contamos con una metodología clara para solucionar el problema de los Nan en los target.

##### Red con pérdida grad loss:

In [21]:
def grad_loss_1(v_gt, v):
    # Gradient loss

    res=v-v_gt
    loss = tf.reduce_mean(tf.abs(res), axis=[0,1,2]) # modificada   
    print("res: ",res.shape)
    #tf.reduce_mean(tf.boolean_mask(v - v_gt, tf.is_finite(v-v_gt)), axis=[0,1,2]) 

    jy = v[:, 0, :, :] - v[:, :, :, :] # modificada
    #jy = v[:,1:,:,:] - v[:,:-1,:,:]
    jx = v[:,:,1:,:] - v[:,:,:-1,:]
    jy_ = v_gt[:,0,:,:] - v_gt[:,:,:,:] # modificada
    #jy_ = v_gt[:, 1:, :, :] - v_gt[:,:-1, :, :]
    jx_ = v_gt[:,:,1:,:] - v_gt[:,:,:-1,:]

    #HACER EL MISMO CAMBIO QUE ARRIBA
    loss += tf.reduce_mean(tf.abs(jy - jy_), axis=[0,1,2]) # modificada
    loss += tf.reduce_mean(tf.abs(jx - jx_), axis=[0,1,2]) # modificada
    print('loss:',loss.shape,loss)

    #filtrar valores NAn  

    return loss

In [22]:
def grad_loss_2(v_gt, v):
    # Gradient loss
    print("v: ",v)
    print("##########")
    print("v_gt: ",v_gt)

    loss = tf.reduce_mean(tf.abs(v - v_gt), axis=[0,1,2]) 
    print(loss.shape)
    res=v-v_gt
    print("res: ",res.shape)
    finite=tf.math.is_finite(res)
    print("finite: ",finite.shape)
    mask=tf.boolean_mask(v - v_gt, finite )
    mask=tf.reshape(mask, [1,32,32,1])
    print("type mask: ",type(mask))
    print("mask: ",mask.shape)
    loss = tf.reduce_mean(mask, axis=[0,1]) 
    print("second loss: ",loss.shape)

    jy = v[:, 0, :, :] - v[:, :, :, :] # modificada
    jy = v[:,1:,:,:] - v[:,:-1,:,:]
    jx = v[:,:,1:,:] - v[:,:,:-1,:]
    jy_ = v_gt[:,0,:,:] - v_gt[:,:,:,:] # modificada
    jy_ = v_gt[:, 1:, :, :] - v_gt[:,:-1, :, :]
    jx_ = v_gt[:,:,1:,:] - v_gt[:,:,:-1,:]


    #antes
    #loss += tf.reduce_mean(tf.abs(jy - jy_), axis=[0,1,2]) # modificada
    #loss += tf.reduce_mean(tf.abs(jx - jx_), axis=[0,1,2]) # modificada

    #modificado
    loss += tf.reduce_mean(tf.reshape( tf.boolean_mask(jy - jy_, tf.math.is_finite(jy-jy_)), [1,32,32,1]), axis=[0,1])
    print("Pasó el primero")
    loss += tf.reduce_mean(tf.reshape( tf.boolean_mask(jx - jx_, tf.math.is_finite(jx-jx_)), [1,32,32,1]), axis=[0,1])#, keepdims=True) 
    print('loss:',loss.shape,loss)
    print("Vuelta terminada")

    #filtrar valores NAn  ¿?

    return loss

In [23]:
def grad_loss_3(v_gt,v):

    # Gradient loss
    print("v: ",v)
    print("##########")
    print("v_gt: ",v_gt)

    if v.shape[0] == None:
        
        loss = tf.reduce_mean(tf.abs(v - v_gt), axis=[0,1,2]) 
        print(loss.shape)
        res=v-v_gt
        print("res: ",res.shape)
        finite=tf.math.is_finite(res)
        print("finite: ",finite.shape)
        mask=tf.boolean_mask(v - v_gt, finite )
        mask=tf.reshape(mask, [1,32,32,1])
        print("type mask: ",type(mask))
        print("mask: ",mask.shape)
        loss = tf.reduce_mean(mask, axis=[0,1]) 
        print("second loss: ",loss.shape)

        jy = v[:, 0, :, :] - v[:, :, :, :] # modificada
        jy = v[:,1:,:,:] - v[:,:-1,:,:]
        jx = v[:,:,1:,:] - v[:,:,:-1,:]
        jy_ = v_gt[:,0,:,:] - v_gt[:,:,:,:] # modificada
        jy_ = v_gt[:, 1:, :, :] - v_gt[:,:-1, :, :]
        jx_ = v_gt[:,:,1:,:] - v_gt[:,:,:-1,:]

        loss += tf.reduce_mean(tf.reshape( tf.boolean_mask(jy - jy_, tf.math.is_finite(jy-jy_)), [1,32,32,1]), axis=[0,1])
        print("Pasó el primero")
        loss += tf.reduce_mean(tf.reshape( tf.boolean_mask(jx - jx_, tf.math.is_finite(jx-jx_)), [1,32,32,1]), axis=[0,1])#, keepdims=True) 
        print('loss:',loss.shape,loss)
        print("Vuelta terminada")

        return loss

    else:
        loss = tf.reduce_mean(tf.abs(v - v_gt), axis=[0,1,2]) 
        print(loss.shape)
        res=v-v_gt
        print("res: ",res.shape)
        finite=tf.math.is_finite(res)
        print("finite: ",finite.shape)
        mask=tf.boolean_mask(v - v_gt, finite )
        #mask=tf.reshape(mask, [1,32,32,1])
        print("type mask: ",type(mask))
        print("mask: ",mask.shape)
        loss = tf.reduce_mean(mask, axis=[0,1,2]) 
        print("second loss: ",loss.shape)

        jy = v[:, 0, :, :] - v[:, :, :, :] # modificada
        jy = v[:,1:,:,:] - v[:,:-1,:,:]
        jx = v[:,:,1:,:] - v[:,:,:-1,:]
        jy_ = v_gt[:,0,:,:] - v_gt[:,:,:,:] # modificada
        jy_ = v_gt[:, 1:, :, :] - v_gt[:,:-1, :, :]
        jx_ = v_gt[:,:,1:,:] - v_gt[:,:,:-1,:]

        loss += tf.reduce_mean( tf.boolean_mask(jy - jy_, tf.math.is_finite(jy-jy_)), axis=[0,1,2])
        print("Pasó el primero")
        loss += tf.reduce_mean( tf.boolean_mask(jx - jx_, tf.math.is_finite(jx-jx_)), axis=[0,1,2])#, keepdims=True) 
        print('loss:',loss.shape,loss)
        print("Vuelta terminada")

        return loss

In [ ]:
model_grad_loss = cnn.get_model(loss=grad_loss_3)
# model = cnn.get_model(residual=True) # RPN
#model_grad_loss.summary()

h_grad_loss=model_grad_loss.fit([inp_tr,time_tr,lat_tr,lon_tr,orog_tr], target_tr, epochs=4,
            validation_data=([inp_va,time_va,lat_va,lon_va,orog_va], target_va))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=[12,3])
plt.plot(h_grad_loss.history['mae'][1:], label='mae')
plt.plot(h_grad_loss.history['val_mae'][1:], label = 'val_mae')
plt.xlabel('Epoch')
plt.ylabel('MAE')
#plt.ylim([0.0, 1])
plt.legend(loc='lower right')
plt.show()

##### 5.2 Red con pérdida mse

In [ ]:
model_mse = cnn.get_model(loss='mse')#, residual=True) # DCN
# model = cnn.get_model(residual=True) # RPN
#model_mse.summary()
h_mse=model_mse.fit([inp_tr,time_tr,lat_tr,lon_tr,orog_tr], target_tr, epochs=4,
            validation_data=([inp_va,time_va,lat_va,lon_va,orog_va], target_va))

5.3 Red con pérdida mae

In [ ]:
model_mae = cnn.get_model(loss='mae')#, residual=True) # DCN
# model = cnn.get_model(residual=True) # RPN
#model_mse.summary()
h_mae=model_mae.fit([inp_tr,time_tr,lat_tr,lon_tr,orog_tr], target_tr, epochs=4,
            validation_data=([inp_va,time_va,lat_va,lon_va,orog_va], target_va))